# Llaves de las diferentes API

In [2]:
TWILIO_ACCOUNT_SID = "adfadfadfadfadf"
TWILIO_AUTH_TOKEN = "adfadfadfadfadf"
PHONE_NUMBER = "adfadfadfadfaddf"
API_KEY_WAPI = "adfadfadfadfadf"

# Instalamos e importamos librerias

In [7]:
#Instalamos Twilio
!pip3 install twilio

You should consider upgrading via the 'c:\users\arnol\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [10]:
import sys
print (sys.version)

3.11.4 (tags/v3.11.4:d2340ef, Jun  7 2023, 05:45:37) [MSC v.1934 64 bit (AMD64)]


In [11]:
#Importamos librerias

import os
from twilio.rest import Client
#from twilio_config import TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN, PHONE_NUMBER, API_KEY_WAPI
import time

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json


import pandas as pd
import requests
from bs4  import BeautifulSoup
from tqdm import tqdm

from datetime import datetime

ModuleNotFoundError: No module named 'twilio'

# Armamos de la URL

In [ ]:
#Query es la ciudad de donde queremos obtener el estado del tiempo y la api_key es la que obtenemos de API Weather
query = 'salama baja verapaz'
api_key = API_KEY_WAPI

#Obtenemos la URL donde hacemos las consultas a la API
url_clima = 'http://api.weatherapi.com/v1/forecast.json?key='+api_key+'&q='+query+'&days=1&aqi=no&alerts=no'
url_clima

'http://api.weatherapi.com/v1/forecast.json?key=bdef0dfd918c47dbbc865913231109&q=salama baja verapaz&days=1&aqi=no&alerts=no'

In [ ]:
#Hacemos la peticion y la convertimos en un json
response = requests.get(url_clima).json()
response

{'location': {'name': 'Salama',
  'region': 'Baja Verapaz',
  'country': 'Guatemala',
  'lat': 15.1,
  'lon': -90.27,
  'tz_id': 'America/Guatemala',
  'localtime_epoch': 1695031613,
  'localtime': '2023-09-18 4:06'},
 'current': {'last_updated_epoch': 1695031200,
  'last_updated': '2023-09-18 04:00',
  'temp_c': 17.5,
  'temp_f': 63.5,
  'is_day': 0,
  'condition': {'text': 'Fog',
   'icon': '//cdn.weatherapi.com/weather/64x64/night/248.png',
   'code': 1135},
  'wind_mph': 2.5,
  'wind_kph': 4.0,
  'wind_degree': 262,
  'wind_dir': 'W',
  'pressure_mb': 1015.0,
  'pressure_in': 29.97,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'humidity': 97,
  'cloud': 100,
  'feelslike_c': 17.5,
  'feelslike_f': 63.6,
  'vis_km': 0.0,
  'vis_miles': 0.0,
  'uv': 1.0,
  'gust_mph': 3.9,
  'gust_kph': 6.3},
 'forecast': {'forecastday': [{'date': '2023-09-18',
    'date_epoch': 1694995200,
    'day': {'maxtemp_c': 26.3,
     'maxtemp_f': 79.3,
     'mintemp_c': 17.2,
     'mintemp_f': 62.9,
     'avgte

## Ruta del JSON

In [ ]:
#Vemos las keys del JSON, para navegar el JSON de una mejor manera
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [ ]:
#Obtenemos la seccion Forecast, y seleccionamos el dia actual
response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [ ]:
#Obtenemos la seccion del campo hora y vemos su longitud con la funcion len
len(response['forecast']['forecastday'][0]['hour'])

24

In [ ]:
#Vemos que lo que nos interesa es la seccion Time, en donde se encuentra la fecha
response['forecast']['forecastday'][0]['hour'][0]

{'time_epoch': 1695016800,
 'time': '2023-09-18 00:00',
 'temp_c': 16.9,
 'temp_f': 62.5,
 'is_day': 0,
 'condition': {'text': 'Mist',
  'icon': '//cdn.weatherapi.com/weather/64x64/night/143.png',
  'code': 1030},
 'wind_mph': 3.1,
 'wind_kph': 5.0,
 'wind_degree': 330,
 'wind_dir': 'NNW',
 'pressure_mb': 1016.0,
 'pressure_in': 30.01,
 'precip_mm': 0.0,
 'precip_in': 0.0,
 'humidity': 97,
 'cloud': 45,
 'feelslike_c': 16.9,
 'feelslike_f': 62.5,
 'windchill_c': 16.9,
 'windchill_f': 62.5,
 'heatindex_c': 16.9,
 'heatindex_f': 62.5,
 'dewpoint_c': 16.4,
 'dewpoint_f': 61.5,
 'will_it_rain': 0,
 'chance_of_rain': 0,
 'will_it_snow': 0,
 'chance_of_snow': 0,
 'vis_km': 2.0,
 'vis_miles': 1.0,
 'gust_mph': 6.3,
 'gust_kph': 10.1,
 'uv': 1.0}

In [ ]:
#Obtenemos unicamente la fecha
response['forecast']['forecastday'][0]['hour'][0]['time'].split()[0]

'2023-09-18'

In [ ]:
#Obtenemos la hora, con el segundo split seleccionamos solo la hora y no los minutos y lo convertimos en numero entero
int(response['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(':')[0])

'01'

In [ ]:
#Navegamos en el JSON y vemos la condicion si llovera o no
response['forecast']['forecastday'][0]['hour'][0]['will_it_rain']

0

In [ ]:
#Navegamos en el JSON y vemos la probabilidad de lluvia
response['forecast']['forecastday'][0]['hour'][2]['chance_of_rain']

0

# Armamos el DataFrame

In [ ]:
#Creamos una funcion para obtener la informacion de la API, segun lo armamos previamente
def get_forecast(response,i):

    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hora = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0])
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    tempe = float(response['forecast']['forecastday'][0]['hour'][i]['temp_c'])
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    prob_rain = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']

    return fecha,hora,condicion,tempe,rain,prob_rain

In [ ]:
#Hacemos un ciclo for para hacer iterar y obtener la informacion de cada hora del dia
#Usamos la libreria tqdm para monitoriar el progreso del ciclo

datos = []

for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])),colour = 'green'):

    #Llevamos el array con los datos
    datos.append(get_forecast(response,i))


100%|██████████| 24/24 [00:00<00:00, 25738.51it/s]


In [ ]:
#Vemos los datos que obtuvimos
datos

[('2023-09-18', 0, 'Mist', 18.3, 0, 0),
 ('2023-09-18', 1, 'Mist', 18.0, 0, 0),
 ('2023-09-18', 2, 'Fog', 17.9, 0, 0),
 ('2023-09-18', 3, 'Mist', 17.7, 0, 0),
 ('2023-09-18', 4, 'Fog', 17.5, 0, 0),
 ('2023-09-18', 5, 'Fog', 17.3, 0, 0),
 ('2023-09-18', 6, 'Fog', 17.2, 0, 0),
 ('2023-09-18', 7, 'Mist', 18.8, 0, 0),
 ('2023-09-18', 8, 'Sunny', 20.7, 0, 0),
 ('2023-09-18', 9, 'Sunny', 22.5, 0, 0),
 ('2023-09-18', 10, 'Sunny', 24.5, 0, 0),
 ('2023-09-18', 11, 'Sunny', 25.7, 0, 0),
 ('2023-09-18', 12, 'Sunny', 24.7, 0, 0),
 ('2023-09-18', 13, 'Sunny', 23.5, 0, 0),
 ('2023-09-18', 14, 'Sunny', 25.1, 0, 0),
 ('2023-09-18', 15, 'Sunny', 26.3, 0, 0),
 ('2023-09-18', 16, 'Patchy rain possible', 25.9, 1, 82),
 ('2023-09-18', 17, 'Patchy rain possible', 24.0, 0, 69),
 ('2023-09-18', 18, 'Patchy rain possible', 20.8, 1, 88),
 ('2023-09-18', 19, 'Patchy rain possible', 19.8, 1, 79),
 ('2023-09-18', 20, 'Patchy rain possible', 19.4, 0, 60),
 ('2023-09-18', 21, 'Partly cloudy', 18.9, 0, 0),
 ('2023-09

In [ ]:
#Creamos el DataFrame
col = ['Fecha','Hora','Condicion','Temperatura','Lluvia','prob_lluvia']
df = pd.DataFrame(datos,columns=col)

#ordenamos el DF por la hora y que sea ascendente
df = df.sort_values(by = 'Hora',ascending = True)
df

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
0,2023-09-18,0,Mist,18.3,0,0
1,2023-09-18,1,Mist,18.0,0,0
2,2023-09-18,2,Fog,17.9,0,0
3,2023-09-18,3,Mist,17.7,0,0
4,2023-09-18,4,Fog,17.5,0,0
5,2023-09-18,5,Fog,17.3,0,0
6,2023-09-18,6,Fog,17.2,0,0
7,2023-09-18,7,Mist,18.8,0,0
8,2023-09-18,8,Sunny,20.7,0,0
9,2023-09-18,9,Sunny,22.5,0,0


In [ ]:
#Colocamos una condicion para que nos muestre los valores cuando llovera y filtramos para un rango de hora especifico
df_rain =  df[(df['Lluvia']==1) & (df['Hora']>6) & (df['Hora']< 22)]

#Reducimos el DF para que solo nos aparezca la hora y que llovera
df_rain = df_rain[['Hora','Condicion']]
df_rain.set_index('Hora', inplace = True)
df_rain

,Condicion
Hora,
16,Patchy rain possible
18,Patchy rain possible
19,Patchy rain possible


In [ ]:
#Armamos el mensaje que llegara a nuestro celular con la informacion que nos interesa
'\nHola! \n\n\n El pronostico del tiempo hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain)

'\nHola! \n\n\n El pronostico del tiempo hoy 2023-09-18 en salama baja verapaz es : \n\n\n                  Condicion\nHora                      \n16    Patchy rain possible\n18    Patchy rain possible\n19    Patchy rain possible'

In [ ]:
#Imprimimos nuestro numero de celular
PHONE_NUMBER

'+12489051970'

# Mensaje desde Twilio

In [ ]:
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID
auth_token = TWILIO_AUTH_TOKEN

client = Client(account_sid, auth_token)

message = client.messages \
                .create(
                     body='\nHola! \n\n\n El pronostico de lluvia hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain),
                     from_=PHONE_NUMBER,
                     to='+50232553245'
                 )

print('Mensaje Enviado ' + message.sid)

Mensaje Enviado SM52955dc21e1ed0931c59aee57fe06272
